In [4]:
import pandas as pd
import numpy as np
import pickle
import torch
import random

In [2]:
MAX_COND_SEQ=56#[118,201,248,248]
MAX_PROC_SEQ=40
MAX_MED_SEQ=819
MAX_LAB_SEQ=899
MAX_BMI_SEQ=118
PATH='C:/Users/mehak/OneDrive - University of Delaware - o365/Beheshti, Rahmat - Mehak - Brennan/model/data/'

In [2]:
def create_vocab(file):
    with open (PATH+file, 'rb') as fp:
        condVocab = pickle.load(fp)
    condVocabDict={}
    condVocabDict['<PAD>']=0
    for val in range(len(condVocab)):
        condVocabDict[condVocab[val]]= val+1    

    return condVocabDict

In [ ]:
def create_age_vocab():
    ageVocabDict={}
    ageVocabDict['<PAD>']=0
    i=1
    for val in range(min_age,max_age+1):
        ageVocabDict[val]= i
        i=i+1
    with open('ageVocabDict', 'wb') as fp:
        pickle.dump(ageVocabDict, fp)

In [29]:
def create_batches(batch_size):
    with open (PATH+'hadmDic', 'rb') as fp:
        hids = pickle.load(fp)
    
    batchDict={}
    with open (PATH+'dataDic', 'rb') as fp:
        dataDic = pickle.load(fp)

    batch_idx=0
    for i in range(0,len(hids),batch_size):
        rids=random.sample(range(0,len(hids)), batch_size)
        batch_hids=hids[rids]
        batchDict[batch_idx]=dict((k, dataDic[k]) for k in batch_hids)
        batch_idx=batch_idx+1
        
    return batchDict

In [ ]:
def init(batch_size,dic=False):
    if dic:
        condVocabDict=create_vocab('condVocab')
        procVocabDict=create_vocab('procVocab')
        medVocabDict=create_vocab('medVocab')
        
#         create_age_vocab()
        with open(PATH+'condVocabDict', 'wb') as fp:
            pickle.dump(condVocabDict, fp)
        with open(PATH+'procVocabDict', 'wb') as fp:
            pickle.dump(procVocabDict, fp)
        with open(PATH+'medVocabDict', 'wb') as fp:
            pickle.dump(medVocabDict, fp)

        
    batchDict= create_batches(batch_size)
    
    with open(PATH+'batchDict', 'wb') as fp:
        pickle.dump(batchDict, fp)
    #print(batchDict[1])
    return len(batchDict)

In [ ]:
def get_batches():
    with open (PATH+'batchDict', 'rb') as fp:
        batchDict = pickle.load(fp)
    return batchDict

In [5]:
#with open (PATH+'batchDict', 'rb') as fp:
#        batchDict = pickle.load(fp)

In [9]:
#get_batch_data(batchDict[1])

Cond
[664, 71, 66, 97, 57, 120, 663, 35, 676]
Proc


TypeError: list indices must be integers or slices, not str

In [8]:
def get_batch_data(data): 
    conds=[]
    cond_lengths=[]    
    cond_ages=[]
    cond_flag=[]
    procs=[]
    proc_lengths=[]    
    proc_ages=[]
    meds=[]
    med_lengths=[]    
    med_ages=[]
    labs=[]
    lab_lengths=[]    
    lab_ages=[]
    bmi=[]
    bmi_lengths=[]    
    bmi_ages=[]
    labels=[]
    census=[]
    with open (PATH+'condVocabDict', 'rb') as fp:
        condVocabDict = pickle.load(fp)
    with open (PATH+'procVocabDict', 'rb') as fp:
        procVocabDict = pickle.load(fp)
    with open (PATH+'medVocabDict', 'rb') as fp:
        medVocabDict = pickle.load(fp)


    for hid, hid_data in data.items():
        for feature, feat_data in hid_data.items():
            #print(feature)
            if feature=='Cond':
                fids=list(pd.Series(feat_data['fids']).map(condVocabDict))
                fids_pad=list(np.zeros(MAX_COND_SEQ))
                fids_pad[0:len(fids)]=fids
                conds.append(fids_pad) 
                cond_lengths.append(len(fids))#(sum(x > 0 for x in fids_pad))
                #print(fids)

            if feature=='Proc':
                fids=list(pd.Series(feat_data['fids']).map(procVocabDict))
                fids_pad=list(np.zeros(MAX_PROC_SEQ))
                fids_pad[0:len(fids)]=fids
                procs.append(fids_pad) 
                proc_lengths.append(len(fids))#(sum(x > 0 for x in fids_pad))
 
            if feature=='Med':
                fids=list(pd.Series(feat_data['fids']).map(medVocabDict))
                fids_pad=list(np.zeros(MAX_MED_SEQ))
                fids_pad[0:len(fids)]=fids
                meds.append(fids_pad) 
                med_lengths.append(len(fids))#(sum(x > 0 for x in fids_pad))
                
        labels.append(hid_data['label'])
    return conds,cond_lengths,procs,proc_lengths,meds,med_lengths,labels